Adapted from https://developers.google.com/optimization/routing/tsp#search_strategy

In [1]:
import pandas as pd

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

**Note**: Since the routing solver does all computations with integers, the distance callback must return an integer distance for any two locations. If any of the entries of data['distance_matrix'] are not integers, you need to round either the matrix entries, or the return values of the callback, to integers. See Scaling the distance matrix for an example that shows how to avoid problems caused by rounding error.

In [2]:
cities = {0: 'New York',
          1: 'Los Angeles',
          2: 'Chicago',
          3: 'Minneapolis',
          4: 'Denver',
          5: 'Dallas',
          6: 'Seattle',
          7: 'Boston',
          8: 'San Francisco',
          9: 'St. Louis',
          10: 'Houston',
          11: 'Phoenix',
          12: 'Salt Lake City',}

In [3]:
city_data = {'New York':(40.7128,-74.0060),
          'Los Angeles':(34.0522,-118.2437),
          'Chicago':(41.8781,-87.6298),
          'Minneapolis':(44.9778,-93.2650),
          'Denver':(39.7392,-104.9903),
          'Dallas':(32.7767,-96.7970),
          'Seattle':(47.6062,-122.3321),
          'Boston':(42.3601,-71.0589),
          'San Francisco':(37.7749,-122.4194),
          'St. Louis':(38.6270,-90.1994),
          'Houston':(29.7604,-95.3698),
          'Phoenix':(33.4484,-112.0740),
          'Salt Lake City':(40.7608,-111.8910),}

In [4]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [5]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [6]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [7]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

### First solution strategy
(taken from ORTools documentation https://developers.google.com/optimization/routing/routing_options)

The first solution strategy is the method the solver uses to find an initial solution. The following table lists the options for first_solution_strategy.

|Option	|Description|
|---|---|
|AUTOMATIC |	Lets the solver detect which strategy to use according to the model being solved.|
|PATH_CHEAPEST_ARC |	Starting from a route "start" node, connect it to the node which produces the cheapest route segment, then extend the route by iterating on the last node added to the route.|
|PATH_MOST_CONSTRAINED_ARC |	Similar to PATH_CHEAPEST_ARC, but arcs are evaluated with a comparison-based selector which will favor the most constrained arc first. To assign a selector to the routing model, use the method ArcIsMoreConstrainedThanArc().|
|EVALUATOR_STRATEGY |	Similar to PATH_CHEAPEST_ARC, except that arc costs are evaluated using the function passed to SetFirstSolutionEvaluator().|
|SAVINGS |	Savings algorithm (Clarke & Wright). Reference: Clarke, G. & Wright, J.W.: "Scheduling of Vehicles from a Central Depot to a Number of Delivery Points", Operations Research, Vol. 12, 1964, pp. 568-581.|
|SWEEP |	Sweep algorithm (Wren & Holliday). Reference: Anthony Wren & Alan Holliday: Computer Scheduling of Vehicles from One or More Depots to a Number of Delivery Points Operational Research Quarterly (1970-1977), Vol. 23, No. 3 (Sep., 1972), pp. 333-344.|
|CHRISTOFIDES |	Christofides algorithm (actually a variant of the Christofides algorithm using a maximal matching instead of a maximum matching, which does not guarantee the 3/2 factor of the approximation on a metric travelling salesperson). Works on generic vehicle routing models by extending a route until no nodes can be inserted on it. Reference: Nicos Christofides, Worst-case analysis of a new heuristic for the travelling salesman problem, Report 388, Graduate School of Industrial Administration, CMU, 1976.|
|ALL_UNPERFORMED |	Make all nodes inactive. Only finds a solution if nodes are optional (are element of a disjunction constraint with a finite penalty cost).|
|BEST_INSERTION |	Iteratively build a solution by inserting the cheapest node at its cheapest position; the cost of insertion is based on the global cost function of the routing model. As of 2/2012, only works on models with optional nodes (with finite penalty costs).|
|PARALLEL_CHEAPEST_INSERTION |	Iteratively build a solution by inserting the cheapest node at its cheapest position; the cost of insertion is based on the arc cost function. Is faster than BEST_INSERTION.|
|LOCAL_CHEAPEST_INSERTION |	Iteratively build a solution by inserting each node at its cheapest position; the cost of insertion is based on the arc cost function. Differs from PARALLEL_CHEAPEST_INSERTION by the node selected for insertion; here nodes are considered in their order of creation. Is faster than PARALLEL_CHEAPEST_INSERTION.|
|GLOBAL_CHEAPEST_ARC |	Iteratively connect two nodes which produce the cheapest route segment.|
|LOCAL_CHEAPEST_ARC |	Select the first node with an unbound successor and connect it to the node which produces the cheapest route segment.|
|FIRST_UNBOUND_MIN_VALUE |	Select the first node with an unbound successor and connect it to the first available node. This is equivalent to the CHOOSE_FIRST_UNBOUND strategy combined with ASSIGN_MIN_VALUE (cf. constraint_solver.h).|

In [8]:
strategy = {'PATH_CHEAPEST_ARC':routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
            'PATH_MOST_CONSTRAINED_ARC':routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC,
            'CHRISTOFIDES':routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,
            #'BEST_INSERTION':routing_enums_pb2.FirstSolutionStrategy.BEST_INSERTION,
            'LOCAL_CHEAPEST_INSERTION':routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION,
            'LOCAL_CHEAPEST_ARC':routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC,
           }

In [9]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

### Local search options
The following table lists the options for local search strategies (also called metaheuristics). See Changing the search strategy for examples of setting these options.

|Option	|Description|
|---|---|
|AUTOMATIC|	Lets the solver select the metaheuristic.|
|GREEDY_DESCENT|	Accepts improving (cost-reducing) local search neighbors until a local minimum is reached.|
|GUIDED_LOCAL_SEARCH|	Uses guided local search to escape local minima (cf. http://en.wikipedia.org/wiki/Guided_Local_Search); this is generally the most efficient metaheuristic for vehicle routing.|
|SIMULATED_ANNEALING|	Uses simulated annealing to escape local minima (cf. http://en.wikipedia.org/wiki/Simulated_annealing).|
|TABU_SEARCH|	Uses tabu search to escape local minima (cf. http://en.wikipedia.org/wiki/Tabu_search).|
|GENERIC_TABU_SEARCH |Uses tabu search on the objective value of solution to escape local minima.|

In [10]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [11]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 7293 miles
Route for vehicle 0:
 0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 -> 9 -> 0



In [12]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES)

In [13]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 7569 miles
Route for vehicle 0:
 0 -> 7 -> 2 -> 3 -> 6 -> 8 -> 1 -> 11 -> 12 -> 4 -> 5 -> 10 -> 9 -> 0



In [14]:
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
    routes.append(route)
    return routes

In [15]:
routes = get_routes(solution, routing, manager)
# Display the routes.
for i, route in enumerate(routes):
    print('Route', i, route)

Route 0 [0, 7, 2, 3, 6, 8, 1, 11, 12, 4, 5, 10, 9, 0]


In [16]:
import plotly.express as px
from ipywidgets import interact

@interact
def run_alg(Heuristic = strategy.keys()):
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (strategy[Heuristic])
    solution = routing.SolveWithParameters(search_parameters)
    print_solution(manager, routing, solution)
    route = get_routes(solution, routing, manager)[0]
    city = [cities[i] for i in route]
    lat = [city_data[i][0] for i in city]
    long = [city_data[i][1] for i in city]
    route_data = pd.DataFrame({'City':city, 'Latitude':lat, 'Longitude':long})
    fig = px.line_geo(route_data, lat = 'Latitude', lon = 'Longitude', text = 'City',
                         scope = 'usa'
                        )
    fig.show()

interactive(children=(Dropdown(description='Heuristic', options=('PATH_CHEAPEST_ARC', 'PATH_MOST_CONSTRAINED_A…

## From (Lat, Long)

In [17]:
text = """
Alabama,Montgomery,32.377716,-86.300568,
Arizona,Phoenix,33.448143,-112.096962,
Arkansas,Little Rock,34.746613,-92.288986,
California,Sacramento,38.576668,-121.493629,
Colorado,Denver,39.739227,-104.984856,
Connecticut,Hartford,41.764046,-72.682198,
Delaware,Dover,39.157307,-75.519722,
Florida,Tallahassee,30.438118,-84.281296,
Georgia,Atlanta,33.749027,-84.388229,
Idaho,Boise,43.617775,-116.199722,
Illinois,Springfield,39.798363,-89.654961,
Indiana,Indianapolis,39.768623,-86.162643,
Iowa,Des Moines,41.591087,-93.603729,
Kansas,Topeka,39.048191,-95.677956,
Kentucky,Frankfort,38.186722,-84.875374,
Louisiana,Baton Rouge,30.457069,-91.187393,
Maine,Augusta,44.307167,-69.781693,
Maryland,Annapolis,38.978764,-76.490936,
Massachusetts,Boston,42.358162,-71.063698,
Michigan,Lansing,42.733635,-84.555328,
Minnesota,St. Paul,44.955097,-93.102211,
Mississippi,Jackson,32.303848,-90.182106,
Missouri,Jefferson City,38.579201,-92.172935,
Montana,Helena,46.585709,-112.018417,
Nebraska,Lincoln,40.808075,-96.699654,
Nevada,Carson City,39.163914,-119.766121,
New Hampshire,Concord,43.206898,-71.537994,
New Jersey,Trenton,40.220596,-74.769913,
New Mexico,Santa Fe,35.68224,-105.939728,
North Carolina,Raleigh,35.78043,-78.639099,
North Dakota,Bismarck,46.82085,-100.783318,
New York,Albany,42.652843,-73.757874,
Ohio,Columbus,39.961346,-82.999069,
Oklahoma,Oklahoma City,35.492207,-97.503342,
Oregon,Salem,44.938461,-123.030403,
Pennsylvania,Harrisburg,40.264378,-76.883598,
Rhode Island,Providence,41.830914,-71.414963,
South Carolina,Columbia,34.000343,-81.033211,
South Dakota,Pierre,44.367031,-100.346405,
Tennessee,Nashville,36.16581,-86.784241,
Texas,Austin,30.27467,-97.740349,
Utah,Salt Lake City,40.777477,-111.888237,
Vermont,Montpelier,44.262436,-72.580536,
Virginia,Richmond,37.538857,-77.43364,
Washington,Olympia,47.035805,-122.905014,
West Virginia,Charleston,38.336246,-81.612328,
Wisconsin,Madison,43.074684,-89.384445,
Wyoming,Cheyenne,41.140259,-104.820236
"""

temp = text.split(',')

state = [temp[i].strip() for i in range(0,192, 4)]
city  = [temp[i] for i in range(1, 192, 4)]
lat   = [float(temp[i]) for i in range(2, 192, 4)]
long  = [float(temp[i]) for i in range(3, 192, 4)]

city_data = pd.DataFrame({'City':city, 'Latitude':lat, 'Longitude':long})

In [18]:
from math import radians, sin, cos, acos

def hav_dist(lat_1, lon_1, lat_2, lon_2):
    lat_1, lon_1 = radians(lat_1), radians(lon_1)
    lat_2, lon_2 = radians(lat_2), radians(lon_2)
    return 3958.76 * acos(sin(lat_1)*sin(lat_2) + cos(lat_1)*cos(lat_2)*cos(lon_1 - lon_2))

In [19]:
def create_data_model_ll():
    """Stores the data for the problem."""
    data = {}
    # Locations in block units
    data['locations'] = [(lat[i], long[i]) for i, _ in enumerate(lat)]
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def compute_hav_distance_matrix(locations):
    """Creates callback to return distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # haversine distance
                distances[from_counter][to_counter] = (int(hav_dist(from_node[0], from_node[1], to_node[0], to_node[1])))
    return distances

data_ll = create_data_model_ll()
distance_matrix_ll = compute_hav_distance_matrix(data_ll['locations'])



In [20]:
import plotly.express as px
from ipywidgets import interact

@interact
def run_alg(Heuristic = strategy.keys()):
    manager = pywrapcp.RoutingIndexManager(len(data_ll['locations']),
                                           data_ll['num_vehicles'], data_ll['depot'])
    
    def distance_callback_ll(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix_ll[from_node][to_node]
    
    routing = pywrapcp.RoutingModel(manager)
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback_ll)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = strategy[Heuristic]
    
    solution = routing.SolveWithParameters(search_parameters)
    
    print_solution(manager, routing, solution)
    route = get_routes(solution, routing, manager)[0]
    ct = [city[i] for i in route]
    lt = [lat[i] for i in route]
    lg = [long[i] for i in route]
    route_data = pd.DataFrame({'City':ct, 'Latitude':lt, 'Longitude':lg})
    fig = px.line_geo(route_data, lat = 'Latitude', lon = 'Longitude', text = 'City',
                         scope = 'usa'
                        )
    fig.show()

interactive(children=(Dropdown(description='Heuristic', options=('PATH_CHEAPEST_ARC', 'PATH_MOST_CONSTRAINED_A…